In [2]:
from scholarly import scholarly
import pandas as pd
import pprint
import time 



In [35]:
import string

def remove_punctuation(input_string):
    # Make a translation table that maps all punctuation characters to None
    translator = str.maketrans("", "", string.punctuation)

    # Apply the translation table to the input string
    result = input_string.translate(translator)

    return result

In [34]:


papers = pd.read_csv("../data/arxiv_papers_full_v2.csv")
try:
    for index, paper in papers.iterrows():

        title = paper["Title"]
        print(title)
        title_with_first_word_removed = remove_punctuation(title).partition(' ')[2]
        search_query = scholarly.search_pubs(title_with_first_word_removed)
        scholarly_result = next(search_query, None)
        if scholarly_result and title.lower() == scholarly_result["bib"]["title"].lower():
            paper["Citations"] = scholarly_result["num_citations"]
        else:
            pass

        time.sleep(3)
except:
    pass

papers.to_csv("../data/arxiv_papers_full_v2_w_citations.csv")  

Natural Language Processing using Hadoop and KOSHIK


PermissionError: [Errno 13] Permission denied: '../data/arxiv_papers_full_v2_w_citations.csv'

In [33]:
papers.head()

,Unnamed: 0,Title,PDF URL,Author,DOI,Published Date,Summary,Journal Ref,Primary Category,Category,Entry ID
0,0,Natural Language Processing using Hadoop and K...,http://arxiv.org/pdf/1608.04434v1,"[arxiv.Result.Author('Emre Erturk'), arxiv.Res...",NaN,2016-08-15 23:09:21+00:00,"Natural language processing, as a data analyti...",NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1608.04434v1
1,1,Integrating AI Planning with Natural Language ...,http://arxiv.org/pdf/2202.07138v2,"[arxiv.Result.Author('Kebing Jin'), arxiv.Resu...",NaN,2022-02-15 02:19:09+00:00,Natural language processing (NLP) aims at inve...,NaN,cs.AI,"['cs.AI', 'cs.CL']",http://arxiv.org/abs/2202.07138v2
2,2,Simple Natural Language Processing Tools for D...,http://arxiv.org/pdf/1906.11608v2,[arxiv.Result.Author('Leon Derczynski')],NaN,2019-06-27 13:15:12+00:00,This technical note describes a set of baselin...,NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1906.11608v2
3,3,Towards the Study of Morphological Processing ...,http://arxiv.org/pdf/2006.16212v1,"[arxiv.Result.Author('Mirinso Shadang'), arxiv...",NaN,2020-06-29 17:24:09+00:00,There is no or little work on natural language...,In proceeding of Regional International Confer...,cs.CL,['cs.CL'],http://arxiv.org/abs/2006.16212v1
4,4,Natural Language Understanding with Distribute...,http://arxiv.org/pdf/1511.07916v1,[arxiv.Result.Author('Kyunghyun Cho')],NaN,2015-11-24 23:23:13+00:00,This is a lecture note for the course DS-GA 30...,NaN,cs.CL,"['cs.CL', 'stat.ML']",http://arxiv.org/abs/1511.07916v1


In [2]:
from semanticscholar import SemanticScholar
import csv
sch = SemanticScholar()

papers = pd.read_csv("../data/arxiv_papers_full_v2.csv")
citations = []

already_existing = pd.read_csv("../data/citations.csv")

already_exisiting_list = list(already_existing["PDF URL"])

with open('../data/citations.csv', mode='a') as citations_file:
    citations_writer = csv.writer(citations_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for index, paper in papers.iterrows():
        if paper["PDF URL"] not in already_exisiting_list:
            title = paper["Title"]

            try:
                results = sch.search_paper(title)
            except:
                print('waiting')
                time.sleep(60) 
                results = sch.search_paper(title)

            if results:
                if title.lower() == results[0].title.lower():
                    citations_num = results[0]["citationCount"]
                else:
                    citations_num = 0
            else:
                citations_num = 0
            
            citations_writer.writerow([paper["PDF URL"], citations_num])

            print(f'{index} of {len(papers)} - {title} - {citations_num}')
            time.sleep(5)




waiting


KeyboardInterrupt: 

In [1]:
from semanticscholar import SemanticScholar
import pandas as pd

papers = pd.read_csv("../data/arxiv_papers_full_v2.csv")

papers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3558 entries, 0 to 3557
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        3558 non-null   int64 
 1   Title             3558 non-null   object
 2   PDF URL           3558 non-null   object
 3   Author            3558 non-null   object
 4   DOI               403 non-null    object
 5   Published Date    3558 non-null   object
 6   Summary           3558 non-null   object
 7   Journal Ref       515 non-null    object
 8   Primary Category  3558 non-null   object
 9   Category          3558 non-null   object
 10  Entry ID          3558 non-null   object
dtypes: int64(1), object(10)
memory usage: 305.9+ KB


In [2]:
from PyPDF2 import PdfReader

def get_full_text(filename):

    text = ""
    reader = PdfReader(filename)
    for page in reader.pages:
        text += page.extract_text()

    return text




In [8]:

from tqdm import tqdm

pdf_folder = '../PDF/'

papers_full_text = []

for index, paper in tqdm(papers.iterrows()):
    filename = paper["PDF URL"].split("/")[4]
    try:
        full_text = get_full_text(pdf_folder + filename + ".pdf")
    except:
        full_text = "ERROR"
    papers_full_text.append(full_text)



265it [04:08,  2.07it/s]c:\Users\dmccaskill\source\repos\SGHacks\SGHacks\venv\lib\site-packages\PyPDF2\_cmap.py:142: PdfReadWarning: Advanced encoding /SymbolSetEncoding not implemented yet
  warnings.warn(
536it [07:44,  1.42it/s]Illegal character in Name Object (b'/ABCDEE+\xb7s\xb2\xd3\xa9\xfa\xc5\xe9')
Illegal character in Name Object (b'/ABCDEE+\xb7s\xb2\xd3\xa9\xfa\xc5\xe9')
760it [10:30,  1.82it/s]FloatObject (b'0.000000000000-5684342') invalid; use 0.0 instead
FloatObject (b'0.000000000000-2842171') invalid; use 0.0 instead
FloatObject (b'0.000000000000-5684342') invalid; use 0.0 instead
1485it [22:27,  2.16s/it]unknown widths : 
[0, IndirectObject(162, 0, 2448874668432)]
1564it [22:55,  2.74it/s]FloatObject (b'0.00-3183727') invalid; use 0.0 instead
1619it [23:26,  1.55it/s]FloatObject (b'0.00-5414185') invalid; use 0.0 instead
1677it [24:16,  1.88it/s]FloatObject (b'0.00-891601') invalid; use 0.0 instead
2111it [31:33,  2.27it/s]FloatObject (b'0.00-10') invalid; use 0.0 instea

In [11]:
len(papers_full_text)

3558

In [12]:
papers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3558 entries, 0 to 3557
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        3558 non-null   int64 
 1   Title             3558 non-null   object
 2   PDF URL           3558 non-null   object
 3   Author            3558 non-null   object
 4   DOI               403 non-null    object
 5   Published Date    3558 non-null   object
 6   Summary           3558 non-null   object
 7   Journal Ref       515 non-null    object
 8   Primary Category  3558 non-null   object
 9   Category          3558 non-null   object
 10  Entry ID          3558 non-null   object
dtypes: int64(1), object(10)
memory usage: 305.9+ KB


In [13]:
papers["Full Text"] = papers_full_text
papers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3558 entries, 0 to 3557
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        3558 non-null   int64 
 1   Title             3558 non-null   object
 2   PDF URL           3558 non-null   object
 3   Author            3558 non-null   object
 4   DOI               403 non-null    object
 5   Published Date    3558 non-null   object
 6   Summary           3558 non-null   object
 7   Journal Ref       515 non-null    object
 8   Primary Category  3558 non-null   object
 9   Category          3558 non-null   object
 10  Entry ID          3558 non-null   object
 11  Full Text         3558 non-null   object
dtypes: int64(1), object(11)
memory usage: 333.7+ KB


In [32]:
papers.to_csv("../data/arxiv_papers_full_v2_w_full_text.csv")

In [19]:
papers = papers.astype({"Full Text": "string"})
papers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3558 entries, 0 to 3557
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        3558 non-null   int64 
 1   Title             3558 non-null   object
 2   PDF URL           3558 non-null   object
 3   Author            3558 non-null   object
 4   DOI               403 non-null    object
 5   Published Date    3558 non-null   object
 6   Summary           3558 non-null   object
 7   Journal Ref       515 non-null    object
 8   Primary Category  3558 non-null   object
 9   Category          3558 non-null   object
 10  Entry ID          3558 non-null   object
 11  Full Text         3558 non-null   string
dtypes: int64(1), object(10), string(1)
memory usage: 333.7+ KB


In [34]:
papers.rename(columns={"Unnamed: 0": "ID"}, inplace=True )
papers.rename(index={"ID": 0}, inplace=True )
papers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3558 entries, 0 to 3557
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                3558 non-null   int64 
 1   Title             3558 non-null   object
 2   PDF URL           3558 non-null   object
 3   Author            3558 non-null   object
 4   DOI               403 non-null    object
 5   Published Date    3558 non-null   object
 6   Summary           3558 non-null   object
 7   Journal Ref       515 non-null    object
 8   Primary Category  3558 non-null   object
 9   Category          3558 non-null   object
 10  Entry ID          3558 non-null   object
 11  Full Text         3558 non-null   string
dtypes: int64(1), object(10), string(1)
memory usage: 361.4+ KB


In [10]:
import pandas as pd

papers = pd.read_csv("../data/arxiv_papers_full_v2_w_full_text.csv")
papers.head()


,Unnamed: 0,ID,Title,PDF URL,Author,DOI,Published Date,Summary,Journal Ref,Primary Category,Category,Entry ID,Full Text
0,ID,0,Natural Language Processing using Hadoop and K...,http://arxiv.org/pdf/1608.04434v1,"[arxiv.Result.Author('Emre Erturk'), arxiv.Res...",NaN,2016-08-15 23:09:21+00:00,"Natural language processing, as a data analyti...",NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1608.04434v1,Natural Language Processing using Hadoop and ...
1,1,1,Integrating AI Planning with Natural Language ...,http://arxiv.org/pdf/2202.07138v2,"[arxiv.Result.Author('Kebing Jin'), arxiv.Resu...",NaN,2022-02-15 02:19:09+00:00,Natural language processing (NLP) aims at inve...,NaN,cs.AI,"['cs.AI', 'cs.CL']",http://arxiv.org/abs/2202.07138v2,Integrating AI Planning with Natural Language ...
2,2,2,Simple Natural Language Processing Tools for D...,http://arxiv.org/pdf/1906.11608v2,[arxiv.Result.Author('Leon Derczynski')],NaN,2019-06-27 13:15:12+00:00,This technical note describes a set of baselin...,NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1906.11608v2,arXiv:1906.11608v2 [cs.CL] 26 Jul 2019SIMPLE...
3,3,3,Towards the Study of Morphological Processing ...,http://arxiv.org/pdf/2006.16212v1,"[arxiv.Result.Author('Mirinso Shadang'), arxiv...",NaN,2020-06-29 17:24:09+00:00,There is no or little work on natural language...,In proceeding of Regional International Confer...,cs.CL,['cs.CL'],http://arxiv.org/abs/2006.16212v1,arXiv:2006.16212v1 [cs.CL] 29 Jun 2020Toward...
4,4,4,Natural Language Understanding with Distribute...,http://arxiv.org/pdf/1511.07916v1,[arxiv.Result.Author('Kyunghyun Cho')],NaN,2015-11-24 23:23:13+00:00,This is a lecture note for the course DS-GA 30...,NaN,cs.CL,"['cs.CL', 'stat.ML']",http://arxiv.org/abs/1511.07916v1,Natural Language Understanding with\nDistribut...


In [11]:
import string

def process_text(text):
    text = text.replace("\n", "")
    text = text.lower()
    text = remove_punctuation(text)
    text = " ".join(text.split())
    
    return text

import string

def remove_punctuation(input_string):
    # Make a translation table that maps all punctuation characters to None
    translator = str.maketrans("", "", string.punctuation)

    # Apply the translation table to the input string
    result = input_string.translate(translator)

    return result

In [12]:
from tqdm import tqdm

papers["Full Text"] = tqdm(papers.apply(lambda x : process_text(x["Full Text"]), axis=1))
papers["Summary"] = tqdm(papers.apply(lambda x : process_text(x["Summary"]), axis=1))
papers["Title"] = tqdm(papers.apply(lambda x : process_text(x["Title"]), axis=1))



100%|██████████| 3558/3558 [00:00<00:00, 504098.56it/s]


In [13]:
papers.head()

,Unnamed: 0,ID,Title,PDF URL,Author,DOI,Published Date,Summary,Journal Ref,Primary Category,Category,Entry ID,Full Text
0,ID,0,natural language processing using hadoop and k...,http://arxiv.org/pdf/1608.04434v1,"[arxiv.Result.Author('Emre Erturk'), arxiv.Res...",NaN,2016-08-15 23:09:21+00:00,natural language processing as a data analytic...,NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1608.04434v1,natural language processing using hadoop and k...
1,1,1,integrating ai planning with natural language ...,http://arxiv.org/pdf/2202.07138v2,"[arxiv.Result.Author('Kebing Jin'), arxiv.Resu...",NaN,2022-02-15 02:19:09+00:00,natural language processing nlp aims at invest...,NaN,cs.AI,"['cs.AI', 'cs.CL']",http://arxiv.org/abs/2202.07138v2,integrating ai planning with natural language ...
2,2,2,simple natural language processing tools for d...,http://arxiv.org/pdf/1906.11608v2,[arxiv.Result.Author('Leon Derczynski')],NaN,2019-06-27 13:15:12+00:00,this technical note describes a set of baselin...,NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1906.11608v2,arxiv190611608v2 cscl 26 jul 2019simple natura...
3,3,3,towards the study of morphological processing ...,http://arxiv.org/pdf/2006.16212v1,"[arxiv.Result.Author('Mirinso Shadang'), arxiv...",NaN,2020-06-29 17:24:09+00:00,there is no or little work on natural language...,In proceeding of Regional International Confer...,cs.CL,['cs.CL'],http://arxiv.org/abs/2006.16212v1,arxiv200616212v1 cscl 29 jun 2020towards the s...
4,4,4,natural language understanding with distribute...,http://arxiv.org/pdf/1511.07916v1,[arxiv.Result.Author('Kyunghyun Cho')],NaN,2015-11-24 23:23:13+00:00,this is a lecture note for the course dsga 300...,NaN,cs.CL,"['cs.CL', 'stat.ML']",http://arxiv.org/abs/1511.07916v1,natural language understanding withdistributed...


In [18]:
def return_id_string(df, title):
    subset = df[df["Full Text"].str.contains(title)]
    return subset.Index.tolist()

In [20]:
papers["Citations"] = tqdm(papers.apply(lambda x : return_id_string(x, x["Title"]), axis=1))


AttributeError: 'str' object has no attribute 'str'